**Can be ignored.**


In [ ]:
from utils import get_tickers, download_m1_raw_data
from datetime import date
from polygon.rest import RESTClient
import pandas as pd
import os
import pyarrow.parquet as pq

DATA_PATH = "../data/polygon/"

with open(DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

Some fastparquet files were corrupt for unknown reasons. So I download them again.

In [ ]:
tickers = get_tickers(3)

corrupt = []
files = os.listdir(DATA_PATH + f"raw/m1/")
for file in files:
    try:
        pd.read_parquet(DATA_PATH + f"raw/m1/{file}")
    except:
        corrupt.append(file)
        print(file)

In [ ]:
for file in corrupt:
    id = file[:-8]
    ticker = id[:-11]
    start = tickers.loc[tickers['ID'] == id, 'start_date'].values[0]
    end = tickers.loc[tickers['ID'] == id, 'end_date'].values[0]    
    m1 = download_m1_raw_data(ticker = ticker, from_ = start, to = end, columns = ["open", "high", "low", "close", "volume"], client=client)
    m1.to_parquet(DATA_PATH + f"raw/m1/{id}.parquet", engine="fastparquet", compression="snappy", row_group_offsets=25000)
    print(id)

In [ ]:
fig, axs = plt.subplot_mosaic([['upper left', 'right'],
                               ['upper left', 'right'],
                               ['mid left', 'right'],
                               ['lower left', 'right']],
                              figsize=(7, 5), layout="constrained", gridspec_kw={'width_ratios': [1.6, 1]})
axs['upper left'].plot(daily.index, daily["cumulative_return"]*100, color="midnightblue", linewidth=1)
axs['upper left'].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
axs['upper left'].set_title('Total return', fontsize=10, fontweight='bold')

# Drawdown
axs['mid left'].plot(drawdowns.index, drawdowns*100, color="firebrick", linewidth=1)
axs['mid left'].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
axs['mid left'].set_title('Drawdown', fontsize=10, fontweight='bold')

# Monthly returns
monthly_return = daily['return'].resample('1M').sum()
colors = ['firebrick' if ret < 0 else 'g' for ret in monthly_return]
monthly_return_index = monthly_return.index.values
monthly_return_index[0] = daily.index[0] # To make the x-axis align
monthly_return_index[-1] = daily.index[-1] # To make the x-axis align

axs['lower left'].bar(monthly_return.index, monthly_return.values*100, width=15, color=colors)
axs['lower left'].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
axs['lower left'].set_title('Monthly returns', fontsize=10, fontweight='bold')

# Statistics
rows = ['Annual return', 'Sharpe', 'Sortina', 'Alpha', 'Beta', 'Average profit/trade']
data = ['1', '2', '3', '4', '5', '6']
axs['right'].table(cellText=data, rowLabels=rows, loc='upper center', edges='open', colWidths=[1])
#axs['right'].set_title('Statistics', fontsize=10, fontweight='bold')
axs['right'].axis("off")